## 2 Preprocessing

### 0) Setup

In [20]:
import csv
import cv2
import numpy as np
from pathlib import Path
import shutil
from PIL import Image

CSV_PATHS = [
    "./datasets/Playing-Cards-Images-Object-Detection-Dataset/dataset_converted.csv",
    "./datasets/Playing-Cards-Object-Detection-Dataset/dataset_converted.csv",
    "./datasets/Playing-Cards-Labelized-Dataset/dataset_converted.csv",
    "./datasets/The-Complete-Playing-Card-Dataset/dataset_converted.csv",
    "./datasets/poker-cards-cxcvz/dataset_converted.csv"
]

### 1) Regolazione del contrasto

In [ ]:
# === CONFIGURATION PARAMETERS ===
# Modificare: quando si modificherà il csv dei dataset, fare che lo scorre e prende in automatico
# il path delle immagini con contrasto sotto la soglia
images = [
    "datasets/Playing-Cards-Object-Detection-Dataset/test/images/416724470_jpg.rf.b9479650b738fa89f1eff26bcd5ea65c.jpg",
    "datasets/Playing-Cards-Object-Detection-Dataset/valid/images/349250549_jpg.rf.74ea2da58134f73aa9a3d9f4531ca9bd.jpg",
    "datasets/Playing-Cards-Object-Detection-Dataset/train/images/873020013_jpg.rf.b4818103a66109a0dd85699556ad4ee1.jpg"
]
OUTPUT_DIR = Path('./preprocessing_results/out_contrast')
THRESHOLD = 0.08      # low contrast threshold
METHOD = 'clahe'       # 'clahe', 'stretch', 'both'
CLIP_LIMIT = 3.0
TILE = 50
# =========================================

def image_contrast(L: np.ndarray) -> float:
    p2, p98 = np.percentile(L, (2, 98))
    return (p98 - p2) / 255.0  # relative contrast

def stretch_channel(L: np.ndarray, low: float, high: float) -> np.ndarray:
    L = L.astype(np.float32)
    denom = max(high - low, 1e-6)
    L = (L - low) * (255.0 / denom)
    return np.clip(L, 0, 255).astype(np.uint8)

def apply_contrast(img: np.ndarray, method: str, clip_limit: float, tile: int) -> np.ndarray:
    if img.ndim == 2 or img.shape[2] == 1:
        L = img if img.ndim == 2 else img[..., 0]
        if method in ('stretch', 'both'):
            p2, p98 = np.percentile(L, (2, 98))
            L = stretch_channel(L, p2, p98)
        if method in ('clahe', 'both'):
            clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(tile, tile))
            L = clahe.apply(L)
        return L
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    L, a, b = cv2.split(lab)
    if method in ('stretch', 'both'):
        p2, p98 = np.percentile(arr, (2, 98))
        L = stretch_channel(L, p2, p98)
    if method in ('clahe', 'both'):
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(tile, tile))
        L = clahe.apply(L)
    lab = cv2.merge([L, a, b])
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def process_image(in_path: Path, out_root: Path) -> tuple[bool, float]:
    img = cv2.imread(str(in_path), cv2.IMREAD_UNCHANGED)
    if img is None:
        return (False, 0.0)
    out_path = out_root / Path(in_path).name
    out_root.mkdir(parents=True, exist_ok=True)

    if img.ndim == 2 or (img.ndim == 3 and img.shape[2] == 1):
        L = img if img.ndim == 2 else img[..., 0]
    else:
        L = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)[:, :, 0]

    contrast = image_contrast(L)
    print(f"new contrast: {contrast}")
    if contrast >= THRESHOLD:
        return (False, contrast)

    out = apply_contrast(img, METHOD, CLIP_LIMIT, TILE)
    cv2.imwrite(str(out_path), out)
    return (True, contrast)

# === EXECUTION ===
total = len(images)
corrected = 0

for img_path in images:
    saved, c = process_image(Path(img_path), OUTPUT_DIR)
    if saved:
        corrected += 1
    print(f"{img_path} -> contrasto {c:.3f} {'(corretto)' if saved else '(ok)'}")

print(f"\nImmagini totali: {total}")
print(f"Corrette (contrast < {THRESHOLD}): {corrected}")
# =================

new contrast: 0.2
datasets/Playing-Cards-Object-Detection-Dataset/test/images/416724470_jpg.rf.b9479650b738fa89f1eff26bcd5ea65c.jpg -> contrasto 0.200 (ok)
new contrast: 0.1568627450980392
datasets/Playing-Cards-Object-Detection-Dataset/valid/images/349250549_jpg.rf.74ea2da58134f73aa9a3d9f4531ca9bd.jpg -> contrasto 0.157 (ok)
new contrast: 0.1843137254901961
datasets/Playing-Cards-Object-Detection-Dataset/train/images/873020013_jpg.rf.b4818103a66109a0dd85699556ad4ee1.jpg -> contrasto 0.184 (ok)

Immagini totali: 3
Corrette (contrast < 0.08): 0


### 2) Immagini con dimensioni differenti

In [ ]:
# === SPECIFIC CONFIGURATION PARAMETERS ===
CSV_PATH = CSV_PATHS[2]
TARGET_WIDTH = 1080
TARGET_HEIGHT = 1080
OUTPUT_DIR = Path('./preprocessing_results/out_resize')
# =========================================

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

total = 0
needs_resize = 0
processed = 0
missing = 0

with open(CSV_PATH, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)

    for row in reader:
        total += 1
        img_path = Path(row["image"])
        try:
            w = int(float(row["image_width"]))
            h = int(float(row["image_height"]))
        except Exception:
            print(f"[ERROR] csv dimentions not valid: {img_path}")
            continue

        if w == TARGET_WIDTH and h == TARGET_HEIGHT:
            continue

        if not img_path.exists():
            missing += 1
            print(f"[MISSING] {img_path}")
            continue

        img = cv2.imread(str(img_path))
        if img is None:
            print(f"[ERROR] cannot read {img_path}")
            continue

        resized = cv2.resize(img, (TARGET_WIDTH, TARGET_HEIGHT), interpolation=cv2.INTER_AREA)
        out_path = OUTPUT_DIR / img_path
        out_path.parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(out_path), resized)
        processed += 1
        needs_resize += 1

print("=== RESULTS ===")
print(f"CSV rows read:          {total}")
print(f"To be resized:          {needs_resize}")
print(f"Resized and saved:      {processed}")
print(f"File missing:           {missing}")
print(f"Output in:              {OUTPUT_DIR.resolve()}")


=== RESULTS ===
CSV rows read:          48093
To be resized:          0
Resized and saved:      0
File missing:           0
Output in:              /Users/aliceorlandini/Intelligent-Systems/Intelligent-Systems-Project/out_resize


### 3) Bounding Box
Eliminare le label con bounding box:
- più piccole dell'1% dell'immagine.
- più grandi del 90% dell'immagine.
- fuori dai bordi dell'immagine.
Se l'immagine rimane senza label allora eliminarla tutta.

Cercare di avere una buona distribuzione.

In [4]:
CSV_IN = CSV_PATHS[2]
CSV_OUT = "./preprocessing_results/out_bounding_boxes/new_dataset_converted.csv"
MIN_FRAC = 0.01   # 1%
MAX_FRAC = 0.9    # 90%

def bbox_is_valid(xc, yc, bw, bh, img_w, img_h):
    if bw < img_w * MIN_FRAC or bh < img_h * MIN_FRAC:
        return False
    if bw > img_w * MAX_FRAC or bh > img_h * MAX_FRAC:
        return False
    x1 = xc - bw / 2
    y1 = yc - bh / 2
    x2 = xc + bw / 2
    y2 = yc + bh / 2
    if x1 < 0 or y1 < 0 or x2 > img_w or y2 > img_h:
        return False
    return True


rows_by_image = {}

with open(CSV_IN, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    fieldnames = reader.fieldnames
    for row in reader:
        img_path = row["image"]
        rows_by_image.setdefault(img_path, []).append(row)

kept_rows = []

for img_path, rows in rows_by_image.items():
    valid_rows = []
    for r in rows:
        try:
            xc = float(r["bbox_x_center"]) * float(r["image_width"])
            yc = float(r["bbox_y_center"]) * float(r["image_height"])
            bw = float(r["bbox_width"]) * float(r["image_width"])
            bh = float(r["bbox_height"]) * float(r["image_height"])
            img_w = float(r["image_width"])
            img_h = float(r["image_height"])
        except Exception:
            continue

        if bbox_is_valid(xc, yc, bw, bh, img_w, img_h):
            valid_rows.append(r)

    if valid_rows:
        kept_rows.extend(valid_rows)
    else:
        p = Path(img_path)
        if p.exists():
            p.unlink()
            print(f"[RIMOSSA] {p}")

with open(CSV_OUT, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(kept_rows)

print(f"Immagini iniziali: {len(rows_by_image)}")
print(f"Immagini rimaste:  {len(set(r['image'] for r in kept_rows))}")
print(f"Label rimaste:     {len(kept_rows)}")
print(f"CSV filtrato:      {CSV_OUT}")

Immagini iniziali: 6002
Immagini rimaste:  6000
Label rimaste:     48093
CSV filtrato:      ./preprocessing_results/out_bounding_boxes/new_dataset_converted.csv


### 4) Correggere classi con correlazione troppo grande

### 5) Correzione Label
Controllare se ci sono immagini senza label e il motivo: l'autore del dataset si è sbagliato, oppure perché non c'è alcuna carta. Nel primo caso mettiamo la label, nel secondo cancelliamo l'immagine. 

Inoltre, bisogna controllare anche se il dataset è bilanciato, quindi avere circa lo stesso numero di immagini per ogni label. Questo task è da fare per ultimo perché dipende dai precedenti.

In [ ]:
CSV_PATH = CSV_PATHS[2]
OUT_DIR = Path("./preprocessing_results/out_label_check")

def list_images_without_labels(csv_path):
    """Restituisce immagini che non hanno alcuna label nel CSV."""
    with open(csv_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        images = {}
        for row in reader:
            images.setdefault(row["image"], []).append(row)
    return [img for img, rows in images.items() if len(rows) == 0 or all(not r["class_name"] for r in rows)]

def export_unlabeled_images(csv_path, out_dir):
    """Copia le immagini senza label nella cartella OUT_DIR"""
    out_dir.mkdir(parents=True, exist_ok=True)
    unlabeled = list_images_without_labels(csv_path)
    print(unlabeled)
    for img_path in unlabeled:
        src = Path(img_path)
        if src.exists():
            dst = out_dir / src.name
            shutil.copy2(src, dst)
            print(f"[COPIED] {src} → {dst}")
    print(f"Total suspicious images copied: {len(unlabeled)}")
    return unlabeled

def delete_image_and_csv_row(csv_path, image_path):
    """Deletes an image and removes the corresponding row from the CSV."""
    tmp_csv = csv_path + ".tmp"
    with open(csv_path, newline="", encoding="utf-8") as f_in, open(tmp_csv, "w", newline="", encoding="utf-8") as f_out:
        reader = csv.DictReader(f_in)
        writer = csv.DictWriter(f_out, fieldnames=reader.fieldnames)
        writer.writeheader()
        for row in reader:
            if row["image"] != image_path:
                writer.writerow(row)
    Path(csv_path).unlink()
    Path(tmp_csv).rename(csv_path)
    p = Path(image_path)
    if p.exists():
        p.unlink()
        print(f"[DELETED] {p}")
    print(f"[CSV UPDATED] Removed {image_path}")

def image_stats(img_path: Path):
    try:
        with Image.open(img_path) as im:
            im = im.convert("RGB")
            w, h = im.width, im.height
            gray = np.array(im.convert("L"), dtype=np.float32)
            brightness = float(gray.mean() / 255.0)
            p2, p98 = np.percentile(gray, (2,98))
            contrast = float((p98 - p2) / 255.0)
            return w, h, brightness, contrast
    except Exception:
        return None, None, None, None
    
def add_label(csv_path, image_path, class_name, bbox_xc, bbox_yc, bbox_w, bbox_h):
    """Aggiunge una nuova label calcolando dimensioni, luminosità e contrasto."""
    img = cv2.imread(image_path)
    if img is None:
        print(f"[ERROR] Image not found: {image_path}")
        return
    w, h, brightness, contrast = image_stats(Path(image_path))

    row = {
        "image": image_path,
        "class_name": class_name,
        "bbox_x_center": bbox_xc,
        "bbox_y_center": bbox_yc,
        "bbox_width": bbox_w,
        "bbox_height": bbox_h,
        "image_width": w,
        "image_height": h,
        "brightness": brightness,
        "contrast": contrast,
    }

    # Scrive la nuova riga nel CSV
    with open(csv_path, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=row.keys())
        writer.writerow(row)

    print(f"[ADDED] Label {class_name} for {image_path}")

# === UTILIZZO =============
# 1) Salva nella cartella OUT_DIR le immagini senza label per poterle controllare manualmente
export_unlabeled_images(CSV_PATH, OUT_DIR)

# 2) cancellazione immagine:
# delete_image_and_csv_row(CSV_PATH, "datasets/Playing-Cards-Labelized-Dataset/train/test_unlabeled.jpg")

# 3) ESEMPIO aggiunta label:
# add_label(CSV_PATH, "datasets/Playing-Cards-Labelized-Dataset/train/000000.jpg", "Ah", 0.5, 0.5, 0.4, 0.6)

[]
Total suspicious images copied: 0
[ADDED] Label Ah for datasets/Playing-Cards-Labelized-Dataset/train/000000.jpg
